In [ ]:
# Import library

# Packages required
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import fits
import os
import pandas as pd

from amiral import instructment, utils, parameter, gradient, minimisation, array
from scipy.optimize import minimize 
%matplotlib inline

import tools

from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import least_squares

In [ ]:
# Function

def r0_vs_wvl (wvl):
    return np.power(wvl, 6/5)
    

In [ ]:
# PATH
wdir = "/Users/alau/Repo/amiral_data_analysis/"
# wdir = "/Users/alau/Data/amiral_fits/ganymede_cube/"
_path_csv = ['HD_146233_cube_1_binned_10.csv', 'HD_146233_cube_2_binned_10.csv']
path_csv = ['HD_146233_cube_1_binned_10_FULL_samp.csv', 'HD_146233_cube_2_binned_10_FULL_samp.csv']
# path_csv = ["HD_146233_cube_1_binned_10.csv","HD_146233_cube_2_binned_10.csv"]
amiral_csv = ['sim_2_r0_mu_rho0_free_noise.csv', 'sim_2_r0_mu_rho0_free.csv']
amiral_samp = ['sim_1_r0_samp_free_1.csv', 'sim_2_r0_samp_free.csv']

Read the estimated PSF parameters from the .csv file

In [ ]:
def combine_df (path1, path2): 
    
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path2)
    
    df_list = [df1, df2]
    df = pd.concat(df_list).reset_index(drop=True)
    
    return df

amiral_noise_csv = pd.read_csv(wdir+amiral_csv[0])
amiral_free_csv = pd.read_csv(wdir+amiral_csv[1])
amiral_samp_csv = pd.read_csv(wdir+amiral_samp[1])

amiral_samp_result = combine_df(wdir+amiral_samp[0], wdir+amiral_samp[1])

cube1_csv = pd.read_csv(wdir+_path_csv[0])
cube2_csv = pd.read_csv(wdir+_path_csv[1])

ganymede_frame = [cube1_csv,cube2_csv]
ganymede_result = pd.concat(ganymede_frame).reset_index(drop=True)

# ganymede_result = pd.read_csv(wdir+path_csv[0])

In [ ]:
est_r0 = ganymede_result['r0']
est_sig2 = ganymede_result['amplitude']
wvl = ganymede_result['wvl']

amiral_noise_r0 = amiral_noise_csv['r0']
amiral_r0 = amiral_free_csv['r0']
amiral_samp_r0 = amiral_samp_result['r0']
a = 100*(amiral_samp_r0-est_r0)

print(a)

# k = np.array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
#        4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
#        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
#        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
#        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
#        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
#        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
#        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
#        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
#        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
#        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
#        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

# print(a[0:10])

In [ ]:
n_slice = len(wvl)
true_r0 = []

for i in range (n_slice): 
    _r0 = r0_vs_wvl(wvl[i]*1e-9)
    true_r0.append(_r0)

What is the relationship between r0 and wavelength?

In [ ]:
rcParams['figure.figsize'] = 16,9

fig, ax = plt.subplots()

# ax[0].set_xlim(np.min(true_flux), np.max(true_flux))


Nslice = len(est_r0)

laser_min = 575 -5 # [rjlf] approximative wavelength of the dichroic on MUSE due to LGS
laser_max = 595 +7

index = np.where(est_r0 > 0)[0] # [rjlf] do not account for r0 that where not properly fitted

# [rjlf] Fit all the r0 found with the theoretical curve: r0 ~ wvl^(6/5)
def cost(r):
    return est_r0[index] - r*(wvl[index]/500.*1e9)**(6./5.)

roptim = least_squares(cost,0.13).x[0]

plt.figure(1)
plt.clf()
ax = plt.subplot(111)
plt.plot(wvl[index]*1e9,(roptim*(wvl[index]/500.*1e9)**(6./5.))*100.,color="C1",linewidth=4,label="Theory",zorder=0)
# plt.scatter(wvl[index]*1e9,est_r0[index]*100.,label="Fit",zorder=1,s=25)
plt.xlabel("Lambda [nm]",fontsize=10)
plt.ylabel("r0 [cm]",fontsize=10)

ax.scatter(wvl*1e9,est_r0*100, s = 10, label = 'fitted parameter')
# ax.scatter(wvl*1e9,amiral_samp_r0*100, s = 10, color = 'r', label = 'amiral')

ax.set_ylabel(r'$\mathrm{r_0[cm]}$', fontsize = 14)
ax.set_xlabel(r'$\mathrm{wavelength [nm]}$', fontsize = 14)

ax.legend(loc=1)



In [ ]:
rcParams['figure.figsize'] = 16,9

fig, ax = plt.subplots()

# ax[0].set_xlim(np.min(true_flux), np.max(true_flux))
ax.scatter(wvl*1e9,est_r0*100, s = 10, label = 'fitted parameter')
ax.scatter(wvl*1e9,amiral_r0*100, s = 10, color = 'r', label = 'amiral')

ax.set_ylabel(r'$\mathrm{r_0[cm]}$', fontsize = 14)
ax.set_xlabel(r'$\mathrm{wavelength [nm]}$', fontsize = 14)


ax1 = ax.twinx()
ax1.set_ylabel(r'$\mathrm{k}$', fontsize = 14)
ax1.scatter(wvl*1e9,k, s = 10, color = 'g', label = 'k')
ax.legend(loc=1)
ax1.legend(loc=4)





In [ ]:
rcParams['figure.figsize'] = 16,9
fig, ax = plt.subplots()

# ax[0].set_xlim(np.min(true_flux), np.max(true_flux))

ax.scatter(wvl*1e9,est_r0*100-amiral_samp_r0*100, s = 10)
# ax.scatter(wvl*1e9,amiral_r0*100, s = 10, color = 'r')
ax.set_ylabel(r'$\mathrm{diff(r0)[cm]}$', fontsize = 14)
ax.set_ylim(-2.5,2.5)
ax.set_xlabel(r'$\mathrm{wavelength [nm]}$', fontsize = 14)

In [ ]:
rcParams['figure.figsize'] = 16,9

fig, ax = plt.subplots()
ax.scatter(wvl*1e9,est_sig2, s = 10)
ax.set_ylabel(r'$\mathrm{Variance [rad^2]}$', fontsize = 14)
ax.set_xlabel('$\lambda$ [nm]',fontsize=14)

In [ ]:
rcParams['figure.figsize'] = 16,9

fig, ax = plt.subplots(2,1)

ax[0].scatter(wvl*1e9,est_r0*100, s = 14)
ax[0].set_ylabel(r'$\mathrm{r_0 [m]}$', fontsize = 14)
ax[0].set_xlabel('$\lambda$ [nm]',fontsize=14)

ax[1].scatter(wvl*1e9,est_sig2, s = 14)
ax[1].set_ylabel(r'$\mathrm{sig2 [rad^2]}$', fontsize = 14)
ax[1].set_xlabel('$\lambda$ [nm]',fontsize=14)



In [ ]:
rcParams['figure.figsize'] = 16,9

fig, ax = plt.subplots()

ax.scatter(est_sig2,est_r0,s = 10)
ax.set_xlabel(r'$\mathrm{sig2 [rad^2]}$', fontsize = 14)
ax.set_ylabel(r'$\mathrm{r_0 [m]}$', fontsize = 14)


In [ ]:
index

In [ ]:
from scipy.optimize import least_squares


rcParams['figure.figsize'] = 16,9

est_r0 = amiral_samp_r0


#%% PLOTS
Nslice = len(est_r0)

laser_min = 575 -5 # [rjlf] approximative wavelength of the dichroic on MUSE due to LGS
laser_max = 595 +7

index = np.where(est_r0 > 0)[0] # [rjlf] do not account for r0 that where not properly fitted

# [rjlf] Fit all the r0 found with the theoretical curve: r0 ~ wvl^(6/5)
def cost(r):
    return est_r0[index] - r*(wvl[index]/500.*1e9)**(6./5.)

roptim = least_squares(cost,0.13).x[0]

plt.figure(1)
plt.clf()
ax = plt.subplot(111)
plt.plot(wvl[index]*1e9,(roptim*(wvl[index]/500.*1e9)**(6./5.))*100.,color="C1",linewidth=4,label="Theory",zorder=0)
plt.scatter(wvl[index]*1e9,est_r0[index]*100.,label="Fit",zorder=1,s=25)
plt.xlabel("Lambda [nm]",fontsize=10)
plt.ylabel("r0 [cm]",fontsize=10)
# plt.ylim(5,25)
# plt.xlim(400,950)
#plt.fill([laser_min,laser_max,laser_max,laser_min],[plt.ylim()[0],plt.ylim()[0],plt.ylim()[1],plt.ylim()[1]],color="gray",alpha=.5)
#plt.title("Fried parameter")
plt.legend()
plt.xticks()
plt.yticks()
#plt.fill([laser_min,laser_max,laser_max,laser_min],[plt.ylim()[0],plt.ylim()[0],plt.ylim()[1],plt.ylim()[1]],color="gray",alpha=.5)

print("r0[500nm] = %.2f cm"%(roptim*100))

plt.legend(fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('$\lambda$ [nm]',fontsize=20)
plt.ylabel('$r_0$ [cm]',fontsize=20)


In [ ]:
alpha = ganymede_result['alpha']


rcParams['figure.figsize'] = 16,9

fig, ax = plt.subplots()
ax.scatter(wvl*1e9,alpha,s = 14)
ax.set_xlabel('$\lambda$ [nm]',fontsize=14)
ax.set_ylabel(r'$\alpha$', fontsize = 14)



In [ ]:
beta = ganymede_result['beta']


rcParams['figure.figsize'] = 16,9

fig, ax = plt.subplots()

ax.scatter(wvl*1e9,beta,s = 14)
ax.set_xlabel('$\lambda$ [nm]',fontsize=14)
ax.set_ylabel(r'$\beta$', fontsize = 14)



In [ ]:
alpha 


Plotting the residual

In [ ]:
from maoppy.utils import circavg, circavgplt

wdir = '/Users/alau/Data/MUSE_DATA/HD_146233/'
# fitsname = ['HD_146233_cube_2_binned_10.fits', 'test_2_samp.fits']
fitsname = ['HD_146233_cube_2_binned_10.fits', 'test_2.fits']

In [ ]:
def plt_psf_residual (PSF, FIT, wvl,center = None):
    plt.clf()
    ax.clear()
    
    if center == None: 
        x,y = circavgplt(PSF)
        xm,ym = circavgplt(FIT)
        xp,yp = circavgplt(np.abs(PSF-FIT))
        
    else: 
        x,y = circavgplt(PSF, center = center)
        xm,ym = circavgplt(FIT, center = center)
        xp,yp = circavgplt(np.abs(PSF-FIT), center = center)
        
    plt.semilogy(x,y,label="PSF")
    plt.semilogy(xm,ym,label="MODEL")
    plt.semilogy(xp,(yp),label="|PSF-MODEL|")
    plt.xlabel('Pixel')
    # plt.xlim(-50,50)
#     plt.ylim(1e2,6e8)
    plt.title("wvl = %unm"%(wvl*1e9))
    plt.legend()
    
    plt.draw()
        
    pass

In [ ]:
_result = fits.open(wdir+fitsname[0])
results = _result[1].data

center = (111,105)

results = array.resize_array(results, size = 100, cent=center)

_sim = fits.open(wdir+fitsname[1])
sim_cube = _sim[1].data


In [ ]:
plt.imshow(np.log10(np.abs(results[10]-sim_cube[10])+1e-5))

In [ ]:
center = (111,104)


wvl_start = _result[1].header['CRVAL3']*1e-10 # wvl in metre
wvl_slice = _result[1].header['CD3_3']*1e-10 # wvl in metre

num = 200

plt_psf_residual(results[num], sim_cube[num], wvl = wvl[num], center = center)


In [ ]:
import matplotlib.animation as animation
%matplotlib inline
from IPython.display import HTML

n_img = len(results)

def animate_residual_img (i):
    plt.clf()
    ax.clear()
    
    print("Step %d" %(i))
    
    plt.imshow(np.log10(np.abs(results[i]- sim_cube[i]))+ 1e-5)
    plt.colorbar(label = 'log10(Intensity)')

    plt.title('Residual \n[wvl: %.1f [nm]]' %(wvl[i]*1e9), fontsize = 12)
    plt.draw()

fig = plt.figure()
ax = fig.subplots()

ani = animation.FuncAnimation(fig, animate_residual_img, frames = n_img, interval = 500)

HTML(ani.to_html5_video())

dpi = 300
writer = animation.writers['ffmpeg'](fps=15)
ani.save('residual_img_cube_2.mp4',writer=writer,dpi=dpi)

In [ ]:
def animate_residual (i): 
    print("Step %d" %(i))
    
    plt.clf()
    ax.clear()
    
    if center == None: 
        x,y = circavgplt(results[i])
        xm,ym = circavgplt(sim_cube[i])
        xp,yp = circavgplt(np.abs(results[i]-sim_cube[i]))
        
    else: 
        x,y = circavgplt(results[i], center = center)
        xm,ym = circavgplt(sim_cube[i], center = center)
        xp,yp = circavgplt(np.abs(results[i]-sim_cube[i]), center = center)
        
    plt.semilogy(x,y,label="PSF")
    plt.semilogy(xm,ym,label="MODEL")
    plt.semilogy(xp,(yp),label="|PSF-MODEL|")
    plt.xlabel('Pixel')
    plt.title("wvl = %unm"%(wvl[i]*1e9))
    plt.legend()
    
    plt.draw()
    
fig = plt.figure()
ax = fig.subplots()
ani = animation.FuncAnimation(fig, animate_residual, frames = n_img, interval = 500)
HTML(ani.to_html5_video())

# dpi = 300
# writer = animation.writers['ffmpeg'](fps=15)
# ani.save('residual_cube_samp_2.mp4',writer=writer,dpi=dpi)


In [ ]:
print('End')

In [ ]:
os.getcwd()

In [ ]:
plt.imshow(results_cropped[2]-sim_cube[2])